In [23]:
'''Plan:
1. Load libraries, load data.
2. Preliminary EDA, dealing with missing values.
3. EDA, deleting variables.
4. Feature engineering, ohc.
5. Sample formation.
6. Feature scaling.
7. Model fitting.
8. Performance evaluation.
9. Predictions.
'''

### 1. Load libraries ###

import numpy as np
import pandas as pd
import os, warnings, random, time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from xgboost import XGBClassifier

warnings.filterwarnings("ignore")
#os.getcwd()

In [24]:
### Load data ###

train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
print(train.shape)
train.head(3)

(957919, 120)


,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
0,0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,...,-12.2280,1.7482,1.90960,-7.1157,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,...,-56.7580,4.1684,0.34808,4.1420,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,...,-5.7688,1.2042,0.26290,8.1312,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1


In [25]:
### 2. ###

df = train.sample(n=100000)
#df = train.copy()
df.shape
df.describe()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
count,100000.000000,98423.000000,98436.000000,98387.000000,98409.000000,98416.000000,98412.000000,98324.000000,9.833100e+04,9.838500e+04,...,98392.000000,98374.000000,98390.000000,98419.000000,98349.000000,98408.000000,9.838900e+04,98430.000000,98340.000000,100000.000000
mean,479669.607900,0.090252,0.345467,4045.538415,0.200284,0.304850,-0.067880,1630.612810,3.766018e+05,1.804475e+15,...,-19.866839,2.072613,23.700881,1.767113,62769.411521,1.208571,4.266137e+16,3961.897576,0.558247,0.498240
std,276304.441389,0.043424,0.146654,6375.592514,0.211903,0.145161,2.104665,1275.251136,3.459411e+05,2.332285e+15,...,18.536676,0.893390,45.526179,10.086453,92063.638166,0.114896,6.718075e+16,3160.040257,0.406459,0.499999
min,9.000000,-0.149910,-0.018172,-8598.600000,-0.063067,-0.006924,-12.456000,-189.670000,-2.833700e+04,-1.050800e+15,...,-102.390000,0.346090,-27.691000,-26.276000,-60269.000000,0.907890,-7.347200e+15,-407.920000,-0.121360,0.000000
25%,240558.250000,0.070284,0.282170,414.950000,0.034455,0.240538,-1.115825,493.595000,9.016450e+04,1.212800e+13,...,-28.711000,1.487800,-0.662570,-4.435050,2459.200000,1.146800,2.316300e+14,1306.550000,0.277110,0.000000
50%,480140.000000,0.090210,0.388875,1280.100000,0.136820,0.327420,-0.375790,1468.100000,2.890400e+05,5.055200e+14,...,-14.622500,1.663000,1.688850,0.894340,19172.000000,1.177100,1.335900e+16,3230.900000,0.472325,0.000000
75%,719156.000000,0.116480,0.458260,4435.500000,0.296240,0.413110,0.916892,2506.000000,5.599350e+05,3.102000e+15,...,-5.289925,2.502900,18.520250,6.808850,87905.000000,1.241300,5.252000e+16,6149.500000,0.742993,1.000000
max,957911.000000,0.401770,0.516690,38518.000000,1.285600,0.548840,10.692000,5388.300000,1.886900e+06,1.042400e+16,...,1.310100,4.565900,212.170000,46.715000,510430.000000,1.834000,3.238000e+17,13105.000000,2.690100,1.000000


In [26]:
# are there numerical features?

un_colval = pd.DataFrame([[x,len(df[x].unique())] for x in df.columns], columns = ['colname', 'n_unique'])
un_colval.loc[un_colval.n_unique < 100]
# all columns contain numerical features

,colname,n_unique
119,claim,2


In [27]:
# omitting missing values #

#df.info(verbose=True, null_counts=True)
df = df.dropna()
df.shape

(37423, 120)

In [28]:
### 3. ###

# we have many variables, are all of them useful? lets see correlation

cors = df.corr()
cors.loc[((cors > 0.8)&(cors<1)).any(1)]
# there are no pairwise correlations above 80%

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim


In [29]:
### 4. ###

# check skew and possibly transform some variables #


In [30]:
### 5. ###

y_train = df[['claim']]
X_train = df.drop(columns=['claim'])

print(y_train.shape, X_train.shape)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=2)

print(y_train.shape, X_train.shape, X_test.shape)

(37423, 1) (37423, 119)
(33680, 1) (33680, 119) (3743, 119)


In [31]:
### 6. Scaling ###

ss = StandardScaler()

for i in X_train.columns:
    X_train[[i]] = ss.fit_transform(X_train[[i]])
    X_test[[i]] = ss.transform(X_test[[i]])

X_train.describe()  

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
count,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,...,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04
mean,-4.614941e-17,-1.723367e-16,-2.276847e-16,-6.709136e-17,6.759900e-17,-2.844782e-18,-6.526846e-19,-8.897607e-17,-1.684585e-16,1.585166e-16,...,3.469117e-17,4.014999e-17,4.683259e-16,2.453995e-17,1.165602e-17,1.413491e-17,-1.023653e-15,5.385595e-17,4.349912e-17,5.301249e-17
std,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,...,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00
min,-1.735789e+00,-5.377914e+00,-2.483372e+00,-1.994204e+00,-1.250938e+00,-2.163310e+00,-5.914170e+00,-1.425799e+00,-1.166974e+00,-1.223646e+00,...,-1.450151e+00,-4.443746e+00,-1.926185e+00,-1.131163e+00,-2.738443e+00,-1.138977e+00,-2.559208e+00,-7.435901e-01,-1.380015e+00,-1.657057e+00
25%,-8.660363e-01,-4.612403e-01,-4.195743e-01,-5.645006e-01,-7.877872e-01,-4.352402e-01,-4.988670e-01,-8.925065e-01,-8.301976e-01,-7.676110e-01,...,-9.606197e-01,-4.766457e-01,-6.542054e-01,-5.381229e-01,-6.175693e-01,-6.511689e-01,-5.308956e-01,-6.312502e-01,-8.443441e-01,-6.910875e-01
50%,4.651390e-03,-7.531020e-03,2.968736e-01,-4.295383e-01,-2.977853e-01,1.566639e-01,-1.507298e-01,-1.215875e-01,-2.466854e-01,-5.550956e-01,...,-1.098074e-01,2.800509e-01,-4.597107e-01,-4.861041e-01,-8.722818e-02,-4.740574e-01,-2.710345e-01,-4.371232e-01,-2.344897e-01,-2.052726e-01
75%,8.684124e-01,5.965637e-01,7.664012e-01,5.870339e-02,4.611632e-01,7.397065e-01,4.727733e-01,6.836539e-01,5.259617e-01,5.483510e-01,...,7.524793e-01,7.880920e-01,4.845858e-01,-1.085895e-01,5.034799e-01,2.646904e-01,2.783364e-01,1.428407e-01,6.975473e-01,4.561320e-01
max,1.733535e+00,7.155931e+00,1.162921e+00,5.495634e+00,4.972745e+00,1.666740e+00,5.136039e+00,2.962957e+00,4.372166e+00,3.658337e+00,...,2.391472e+00,1.139968e+00,2.794379e+00,4.063584e+00,4.346456e+00,4.770453e+00,5.461607e+00,4.170953e+00,2.891493e+00,5.204625e+00


In [32]:
### 7. Model fitting ###

time1 = time.time()

log_rg = LogisticRegression()

grid_values = {'penalty': ['l2'], 'C': [0.00001, 0.0001, 0.001, 0.01]}

lr = GridSearchCV(log_rg, param_grid = grid_values, cv=4)
lr.fit(X_train, y_train)

print('logistic', lr.best_score_, lr.best_params_, time.time()-time1)


logistic 0.8619358669833729 {'C': 1e-05, 'penalty': 'l2'} 2.049663782119751


In [ ]:
time1 = time.time()

svm = svm.SVC(kernel='rbf')

grid_values = {'C':[0.01, 0.1, 1, 5, 10]}

svm = GridSearchCV(svm, param_grid = grid_values, cv=2)
svm.fit(X_train, y_train)

print('SVM', svm.best_score_, svm.best_params_, time.time()-time1)

In [33]:
time1 = time.time()

xgbcl = XGBClassifier()

grid_values = {'eta':[0.01, 0.1], 'max_depth':[2,3,5]}

xgb = GridSearchCV(xgbcl, param_grid = grid_values, cv=2)
xgb.fit(X_train, y_train)

print('XGBoost', xgb.best_score_, xgb.best_params_, time.time()-time1)

[18:26:22] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:26:33] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:26:44] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:27:01] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

KeyboardInterrupt: 